In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

# An EDA of Brazilian E-Commerce with OLIST dataset.
---

In recent years, Brazil has seen a surge in online retail activity, transforming the way people shop and interact with businesses. As one of the largest economies in Latin America, understanding the dynamics of Brazilian e-commerce is critical for businesses, policymakers, and researchers alike. This project explores **regional disparities, delivery performance, payment behavior, and customer satisfaction** within the Brazilian e-commerce ecosystem using the **Olist dataset**, a collection of transactional data from an online marketplace.


In [ ]:
# @title
# --- Section 1: Data Load & Cleaning ---
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import requests
import plotly.io as pio

In [ ]:
# @title
# Load datasets
folder = '/content/drive/MyDrive/OLIST SHARED FOLDER/'
orders = pd.read_csv(folder + 'olist_orders_dataset.csv')
customers = pd.read_csv(folder + 'olist_customers_dataset.csv')
sellers = pd.read_csv(folder + 'olist_sellers_dataset.csv')
reviews = pd.read_csv(folder + 'olist_order_reviews_dataset.csv')
payments = pd.read_csv(folder + 'olist_order_payments_dataset.csv')
order_items = pd.read_csv(folder + 'olist_order_items_dataset.csv')
products = pd.read_csv(folder + 'olist_products_dataset.csv')
geolocation = pd.read_csv(folder + 'olist_geolocation_dataset.csv')
category_translation = pd.read_csv(folder + 'product_category_name_translation.csv')

In [ ]:
# @title
# Convert timestamps
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Drop rows with missing customer ID or order ID
orders.dropna(subset=['customer_id', 'order_id'], inplace=True)

# Remove canceled orders (status = "canceled")
orders = orders[orders['order_status'] != 'canceled']

# Merge core datasets into order_full (for most general analysis)
order_customer = pd.merge(orders, customers, on='customer_id', how='left')
order_full = pd.merge(order_customer, order_items, on='order_id', how='left')
order_full = pd.merge(order_full, payments, on='order_id', how='left')
order_full = pd.merge(order_full, reviews[['order_id', 'review_score']], on='order_id', how='left')

# Create helper columns
order_full['purchase_year'] = order_full['order_purchase_timestamp'].dt.year
order_full['purchase_month'] = order_full['order_purchase_timestamp'].dt.month
order_full['delay'] = (order_full['order_delivered_customer_date'] - order_full['order_estimated_delivery_date']).dt.days
order_full['delivery_days'] = (order_full['order_delivered_customer_date'] - order_full['order_purchase_timestamp']).dt.days

# Save initial clean shape
print(" Cleaned Dataset Shape:", order_full.shape)
order_full.head()

# === Build order_df specifically for delivery time vs freight analysis ===

# Merge order_items and products
order_df = order_items.merge(products, on='product_id', how='left')

# Merge orders, keep customer_id to merge with customers next
order_df = order_df.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'customer_id']], on='order_id', how='left')

# Merge customer_state from customers table
order_df = order_df.merge(customers[['customer_id', 'customer_state']], on='customer_id', how='left')

# Merge category translations
order_df = order_df.merge(category_translation, on='product_category_name', how='left')

# Compute delivery time
order_df['delivery_time_days'] = (order_df['order_delivered_customer_date'] - order_df['order_purchase_timestamp']).dt.days

# Final check
print("order_df shape (with customer_state):", order_df.shape)
order_df[['freight_value', 'delivery_time_days', 'customer_state']].head()


In [ ]:
# @title
# ✅ Brazilian state abbreviations to full name
state_abbrev_to_name = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MG': 'Minas Gerais',
    'MS': 'Mato Grosso do Sul',
    'MT': 'Mato Grosso',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'PR': 'Paraná',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina',
    'SE': 'Sergipe',
    'SP': 'São Paulo',
    'TO': 'Tocantins'
}


In [ ]:
# @title
# ✅ Brazilian state population estimates (based on IBGE or approximated)
state_pop = {
    'Acre': 906876,
    'Alagoas': 3351543,
    'Amapá': 877613,
    'Amazonas': 4207714,
    'Bahia': 14930634,
    'Ceará': 9240580,
    'Distrito Federal': 3094325,
    'Espírito Santo': 4064052,
    'Goiás': 7206589,
    'Maranhão': 7075181,
    'Mato Grosso': 3567234,
    'Mato Grosso do Sul': 2839188,
    'Minas Gerais': 21168791,
    'Pará': 8777124,
    'Paraíba': 4039277,
    'Paraná': 11516840,
    'Pernambuco': 9616621,
    'Piauí': 3281480,
    'Rio de Janeiro': 17366189,
    'Rio Grande do Norte': 3534165,
    'Rio Grande do Sul': 11377239,
    'Rondônia': 1815278,
    'Roraima': 652713,
    'Santa Catarina': 7338473,
    'São Paulo': 45919049,
    'Sergipe': 2318822,
    'Tocantins': 1590248
}

## **Exploring Monthly E-Commerce Activity Across Brazilian States**
---

This choropleth map illustrates the number of online orders across Brazilian states for a specific month in the dataset. Each state is shaded according to its order volume, with darker colors representing higher activity levels.

At first glance, São Paulo emerges as the dominant hub of e-commerce, with significantly more orders than other regions. Neighboring states like Minas Gerais and Paraná also show moderate activity, while many northern and interior states appear lighter, indicating lower levels of online shopping during this period.

In [ ]:
# @title
import plotly.express as px
import pandas as pd
import requests

# ✅ GeoJSON for Brazil states
geo_url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
geojson = requests.get(geo_url).json()

# ✅ Add time and geographic info
order_full['year_month'] = pd.to_datetime(order_full['order_purchase_timestamp']).dt.to_period("M").astype(str)
orders_by_state_month = (
    order_full.groupby(['customer_state', 'year_month'])['order_id']
    .nunique()
    .reset_index()
    .rename(columns={'order_id': 'order_count'})
)

orders_by_state_month['state_name'] = orders_by_state_month['customer_state'].map(state_abbrev_to_name)
orders_by_state_month['population'] = orders_by_state_month['state_name'].map(state_pop)
orders_by_state_month['year_month_dt'] = pd.to_datetime(orders_by_state_month['year_month'])
orders_by_state_month = orders_by_state_month.sort_values('year_month_dt')
orders_by_state_month['month_year'] = orders_by_state_month['year_month_dt'].dt.strftime('%b %Y')

# ✅ Vibrant Animated Choropleth
fig = px.choropleth(
    orders_by_state_month,
    geojson=geojson,
    locations='state_name',
    featureidkey='properties.name',
    color='order_count',
    animation_frame='month_year',
    color_continuous_scale='Plasma_r',
    scope='south america',
    hover_name='state_name',
    hover_data={'order_count': True, 'population': True},
    labels={'order_count': 'Order Count'}
)

# ✅ Geos & Style Enhancements
fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.update_layout(
    width=1100,
    height=650,
    margin=dict(l=0, r=0, t=80, b=0),
    title=dict(
        text="<b>How Online Orders Spread Across Brazil</b><br><sub>Monthly Order Count by State (2016–2018)</sub>",
        x=0.5,
        xanchor='center',
        font=dict(size=22, color='black')
    ),
    font=dict(family="Arial", size=14, color='black'),
    coloraxis_colorbar=dict(
        title='Order Count',
        titlefont=dict(size=16, color='black'),
        tickfont=dict(size=12, color='black'),
        ticks="outside",
        ticklen=8,
        tickcolor="black",
        tickwidth=2
    )
)

fig.show()

While the animated choropleth revealed the **geographical spread** of e-commerce over time, this heatmap brings a **different perspective**: it highlights **temporal and regional patterns** in a single compact view.  

Here, I can easily **compare states side by side** and spot those that **lagged in adopting online shopping**. The **white cells** make moments of **complete inactivity** stand out, showing how some regions remained disengaged from e-commerce even as other areas were experiencing rapid growth.  

The heatmap also uncovers **subtle trends** that are harder to see on the map. For example, I can trace how order volumes in **São Paulo climbed steadily** over time, while other states like **Acre** and **Roraima** saw **sporadic bursts of activity** rather than consistent growth. This bird’s-eye view allows me to identify not only **leaders** and **latecomers** but also **periods of stagnation** or **acceleration** across Brazil’s states.


In [ ]:
# @title
import plotly.express as px
import numpy as np
import pandas as pd  # Ensure pandas is imported

# ✅ Prepare heatmap data with full names
heatmap_data = orders_by_state_month.copy()

# ✅ Create user-friendly month labels
heatmap_data['year_month_dt'] = pd.to_datetime(heatmap_data['year_month'])
heatmap_data['year_month_str'] = heatmap_data['year_month_dt'].dt.strftime('%b %Y')

# ✅ Total orders per state for sorting rows
state_order_totals = (
    heatmap_data.groupby('state_name')['order_count']
    .sum()
    .sort_values(ascending=False)
)

# Pivot for heatmap
heatmap_pivot = heatmap_data.pivot_table(
    index='state_name',  # full names
    columns='year_month_dt',  # use datetime for proper sorting
    values='order_count',
    fill_value=0
)

# Sort rows by total orders
heatmap_pivot = heatmap_pivot.loc[state_order_totals.index]

# Replace zeros with NaN to make cells white
heatmap_pivot_alt = heatmap_pivot.replace(0, np.nan)

# ✅ Interactive heatmap
fig2 = px.imshow(
    heatmap_pivot_alt,
    color_continuous_scale='Plasma_r',
    labels=dict(x="Month-Year", y="State", color="Order Count"),
    aspect="auto"
)

# Style cells, grid lines, and layout
fig2.update_traces(
    xgap=1,
    ygap=1,
    hovertemplate='<b>%{y}</b><br>%{x|%b %Y}<br>Orders: %{z}<extra></extra>'
)

# ✅ Layout styling with bold titles and black labels
fig2.update_layout(
    title={
        'text': "<b>Brazilian E-Commerce Orders by State and Month (2016–2018)</b><br><sub>White = No Orders</sub>",
        'x': 0.5,
        'xanchor': 'center',
        'font': dict(size=16, color='black', family='Arial')
    },
    xaxis=dict(
        title="<b>Month-Year</b>",
        titlefont=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black')
    ),
    yaxis=dict(
        title="<b>State</b>",
        titlefont=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black')
    ),
    width=900,
    height=500,
    plot_bgcolor='white'
)

fig2.show()


In [ ]:
# @title
from plotly.subplots import make_subplots
import plotly.graph_objects as go
low_activity_states = [
    'Acre', 'Roraima', 'Amapá', 'Amazonas', 'Rondônia', 'Tocantins',
    'Piauí', 'Maranhão', 'Alagoas', 'Paraíba', 'Sergipe'
]

high_activity_states = [
    'São Paulo', 'Minas Gerais', 'Rio de Janeiro', 'Espírito Santo',
    'Paraná', 'Santa Catarina', 'Rio Grande do Sul',
    'Mato Grosso do Sul', 'Goiás', 'Distrito Federal', 'Mato Grosso'
]

# Step 2: Subset and sort pivoted heatmap data
heatmap_low = heatmap_pivot_alt.loc[heatmap_pivot_alt.index.isin(low_activity_states)].sort_index()
heatmap_high = heatmap_pivot_alt.loc[heatmap_pivot_alt.index.isin(high_activity_states)].sort_index()

# Step 3: Format months for x-axis
month_labels = heatmap_high.columns.strftime('%b %Y')

# Step 4: Create subplot with shared color axis
fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=False,
    column_titles=["<b>Southern and Southeastern States</b>", "<b>Northern and Northeastern States</b>"],
    horizontal_spacing=0.1
)
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(color='black', size=14, family='Arial')

# Step 5: Add high-activity heatmap (left)
fig.add_trace(
    go.Heatmap(
        z=heatmap_high.values,
        x=month_labels,
        y=[f"<b>{state}</b>" for state in heatmap_high.index],
        colorscale='Plasma_r',
        zmin=0,
        zmax=3000,
        xgap=1,
        ygap=1,
        colorbar=dict(title="Order Count", thickness=16, len=0.8),
        hovertemplate='<b>%{y}</b><br>%{x}<br>Orders: %{z}<extra></extra>'
    ),
    row=1, col=1
)

# Step 6: Add low-activity heatmap (right)
fig.add_trace(
    go.Heatmap(
        z=heatmap_low.values,
        x=month_labels,
        y=[f"<b>{state}</b>" for state in heatmap_low.index],
        colorscale='Plasma_r',
        zmin=0,
        zmax=3000,
        xgap=1,
        ygap=1,
        showscale=False,
        hovertemplate='<b>%{y}</b><br>%{x}<br>Orders: %{z}<extra></extra>'
    ),
    row=1, col=2
)

# Step 7: Layout formatting to match your preferred look
fig.update_layout(
    title={
        'text': "<b>Regional Disparities in Brazilian E-Commerce Orders (2016–2018)</b><br><sub>White = No Orders</sub>",
        'x': 0.5,
        'xanchor': 'center',
        'font': dict(size=18, color='black')
    },
    width=1200,
    height=400,
    plot_bgcolor='white',
    margin=dict(t=100),
    xaxis=dict(
        title="<b>Month-Year</b>",
        tickangle=45,
        tickfont=dict(size=11, color='black'),
        titlefont=dict(size=14, color='black')
    ),
    xaxis2=dict(
        title="<b>Month-Year</b>",
        tickangle=45,
        tickfont=dict(size=11, color='black'),
        titlefont=dict(size=14, color='black')
    ),
    yaxis=dict(
        title="<b>State</b>",
        tickfont=dict(size=12, family='Arial', color='black'),
        titlefont=dict(size=14, color='black')
    ),
    yaxis2=dict(
        title="<b>State</b>",
        tickfont=dict(size=12, family='Arial', color='black'),
        titlefont=dict(size=14, color='black')
    )
)

fig.show()


## **Total E-Commerce Order Volume by State**
---

After exploring temporal patterns with the heatmap, I wanted to take a step back and examine the overall distribution of e-commerce activity across Brazilian states. This bar chart shows the total number of unique online orders per state during the entire observation period.

From this visualization, it’s clear that **São Paulo dominates the e-commerce landscape**, with more than 40,000 orders—far surpassing all other states. **Rio de Janeiro** and **Minas Gerais** follow at a distant second and third, while most other states have order volumes that are comparatively small.

This chart highlights the stark regional imbalance in Brazil’s e-commerce market. It emphasizes how heavily concentrated online shopping activity was in a few southeastern states, revealing the long tail of states with minimal engagement. These insights provide important context for understanding the broader dynamics of Brazil’s digital economy.


In [ ]:
# @title
# ✅ Group total number of unique orders by customer state
orders_by_state = (
    order_full.groupby('customer_state')['order_id']
    .nunique()
)

# ✅ Map to full state names
orders_by_state.index = orders_by_state.index.map(state_abbrev_to_name)


In [ ]:
# @title
import plotly.express as px

# ✅ Build dataframe for plotting
orders_by_state_df = pd.DataFrame({
    'State': orders_by_state.index,   # Full state names already mapped earlier
    'Orders': orders_by_state.values
})

# ✅ Create interactive bar chart
fig = px.bar(
    orders_by_state_df,
    x='State',
    y='Orders',
    title='<b>Total E-Commerce Orders by Brazilian State</b>',
    labels={'Orders': 'Number of Unique Orders'},  # y-axis label
    color='Orders',  # Color by order count
    color_continuous_scale='Reds',  # Use red gradient like original matplotlib
    hover_data={'State': True, 'Orders': True}  # Hover details
)

# ✅ Style layout (no text on bars)
fig.update_layout(
    xaxis_tickangle=60,  # Rotate x-axis labels
    width=900,
    height=500,
    plot_bgcolor='white',
    yaxis_gridcolor='lightgray',
    title_font=dict(size=18, family='Arial', color='black'),
    xaxis_title='<b>State</b>',
    yaxis_title='<b>Number of Unique Orders</b>',
    coloraxis_colorbar=dict(title="Total Orders")
)

# ✅ Show interactive chart
fig.show()


## **E-Commerce Engagement Adjusted for Population**
---

While the previous bar chart highlighted **total order volumes**, it didn’t account for differences in **state populations**. To better understand **relative engagement** with online shopping, I calculated the number of **orders per 100,000 residents** for each state. This view helps reveal which regions had the **most active e-commerce participation** on a per-capita basis.

In this visualization, **São Paulo** again leads the ranking, but other patterns emerge. Smaller states like **Espírito Santo**, **Paraíba**, and **Distrito Federal** show surprisingly **high engagement levels** relative to their population sizes. Meanwhile, some highly populated states with large raw order volumes, such as **Bahia** and **Pará**, appear further down the ranking once normalized.

This chart offers a **more balanced perspective**, showing not just where the most orders originated, but where online shopping had the **greatest penetration** among residents. It highlights states that were **highly engaged despite smaller populations** and provides **deeper insights** into Brazil’s e-commerce landscape.


In [ ]:
# @title
import plotly.express as px

# 🛠 Build df (if not already defined)
df = pd.DataFrame({
    'State': orders_by_state.index,
    'Orders': orders_by_state.values,
    'Population': [state_pop[state] for state in orders_by_state.index]
})
df = df.dropna(subset=['Population'])  # Drop rows without population

# 🧮 Normalized orders per 100k people
df['Orders per 100k Residents'] = (df['Orders'] / df['Population']) * 100000

# 📊 Interactive bar plot (normalized)
fig_normalized = px.bar(
    df,
    x='State',
    y='Orders per 100k Residents',
    color='Orders',
    color_continuous_scale='Blues',
    hover_data={
        'Population': True,
        'Orders': True,
        'Orders per 100k Residents': ':.2f'
    },
    title='<b>Normalized E-Commerce Orders: Per 100k Residents by Brazilian State</b>',
    labels={'Orders per 100k Residents': 'Orders per 100k Residents'}
)
fig_normalized.update_layout(
    title={
        'text': '<b>Normalized E-Commerce Orders: Per 100k Residents by Brazilian State</b>',
        'x': 0.5,
        'xanchor': 'center'
    },
    title_font=dict(size=18, family='Arial', color='black'),
    xaxis_title='<b>State</b>',
    yaxis_title='<b>Orders per 100k Residents</b>',
    xaxis_tickangle=60,
    plot_bgcolor='white',
    yaxis_gridcolor='lightgray',
    coloraxis_colorbar=dict(title='Total Orders'),
    width=900,
    height=500,
    xaxis_title_font=dict(color='black'),
    yaxis_title_font=dict(color='black')
)

fig_normalized.show()


## Preferred Payment Methods in Brazilian E-Commerce
---

After exploring regional and temporal patterns in online shopping, I turned my attention to **how Brazilians pay for their online purchases**. This donut chart visualizes the distribution of payment methods across all orders.  

Unsurprisingly, **credit cards dominate the landscape**, accounting for nearly 74% of transactions. This reflects Brazil’s established infrastructure for card-based payments in e-commerce. The second most common method is **boleto**, a popular payment slip system in Brazil, making up about 19% of orders. Smaller segments like **vouchers** and **debit cards** highlight alternative payment behaviors but represent a much smaller share.


In [ ]:
# @title
import plotly.express as px

# Count the number of orders for each payment type
payment_counts = order_full['payment_type'].value_counts().reset_index()
payment_counts.columns = ['payment_type', 'count']

# Custom colors
custom_colors = ['hotpink', 'skyblue', 'gold', 'lightgreen', 'yellow']

# Create donut chart
fig = px.pie(
    payment_counts,
    names='payment_type',
    values='count',
    color_discrete_sequence=custom_colors,
    hole=0.5  # Thinner donut
)

# Center the title and make it bold
fig.update_layout(
    title={
        'text': '<b>Distribution of Payment Methods in Brazil</b>',
        'x': 0.5,
        'xanchor': 'center'
    },
    title_font=dict(size=18, family='Arial', color='black'),
    width=600,   # Smaller width
    height=500   # Smaller height
)

# Improve text inside the donut
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()


While the donut chart showed us the national picture of payment methods, this chart reveals how preferences differ across states. Credit cards still dominate, but in states like **Amazonas and Amapá**, boleto slips are used for nearly 30% of orders.

In [ ]:
# @title
# ✅ Calculate number of orders by state and payment type
payment_counts_by_state = (
    order_full.groupby(['customer_state', 'payment_type'])
    .size()
    .unstack(fill_value=0)
)

# ✅ Convert to percentages
payment_percent_by_state = (
    payment_counts_by_state.div(payment_counts_by_state.sum(axis=1), axis=0) * 100
)

# ✅ Map customer_state abbreviations to full names
payment_percent_by_state.index = payment_percent_by_state.index.map(state_abbrev_to_name)
payment_percent_by_state.index.name = 'state_full'  # So it shows up in reset_index()


In [ ]:
# @title
# ✅ Reset index to bring 'state_full' back as a column
payment_percent_by_state_reset = payment_percent_by_state.reset_index()

# ✅ Melt dataframe for Plotly
payment_melted = payment_percent_by_state_reset.melt(
    id_vars='state_full',
    var_name='Payment Type',
    value_name='Percentage'
)

# ✅ Plot with Plotly
import plotly.express as px

fig = px.bar(
    payment_melted,
    x='state_full',
    y='Percentage',
    color='Payment Type',
    title='<b>Payment Method Distribution by State (Percentage)</b>',
    labels={'state_full': 'State', 'Percentage': 'Percentage of Orders'},
    color_discrete_map={
        'credit_card': 'purple',
        'boleto': 'orange',
        'debit_card': 'green',
        'voucher': 'red'
    }
)

fig.update_layout(
    xaxis_tickangle=60,
    title_font=dict(size=18, family='Arial', color='black'),
    xaxis_title_font=dict(family='Arial Black', size=14, color='black'),  # Bold font for X-axis
    yaxis_title_font=dict(family='Arial Black', size=14, color='black'),  # Bold font for Y-axis
    plot_bgcolor='white',
    yaxis_gridcolor='lightgray',
    width=900,
    height=500
)

fig.show()

This bar chart shows how Brazilians split their online purchases into installments. A clear majority of orders were paid in a single installment, highlighting that many customers prefer to pay upfront. There is a noticeable drop as the number of installments increases, with only a small fraction of orders being divided into higher payments.

In [ ]:
# @title
# Group and get installment counts
installment_counts = order_full['payment_installments'].value_counts().sort_index()

fig1 = px.bar(
    x=installment_counts.index,
    y=installment_counts.values,
    title='<b>Number of Orders by Payment Installment Type</b>',
    labels={'x': 'Number of Installments', 'y': 'Number of Orders'},
    color_discrete_sequence=['coral'],
    width=800,  # Reduced width
    height=500  # Reduced height
)

fig1.update_layout(
    title_font=dict(color='black'),  # Title color
    xaxis_title='<b>Number of Installments</b>',
    yaxis_title='<b>Number of Orders</b>',
    xaxis_title_font=dict(color='black'),  # X-axis title color
    yaxis_title_font=dict(color='black'),  # Y-axis title color
    plot_bgcolor='white',
    yaxis_gridcolor='lightgrey'
)

fig1.show()


States like **Alagoas**, **Paraíba**, and **Rio Grande do Norte** have the **highest average number of installments** per order, suggesting a slightly higher tendency to split payments in these regions. Meanwhile, **São Paulo**, **Distrito Federal**, and **Amapá** show the **lowest averages**. This might reflect differences in **regional income levels**, **credit access**, or **consumer habits** around financing. Overall, Brazilians tend to favor **paying in full** or in **just a few installments**, and the range of state averages (about **2.7 to 4**) shows only **subtle regional differences**.

In [ ]:
# @title
# ✅ Clean + average per order
installments_clean = order_full.drop_duplicates(subset='order_id')
installments_clean = installments_clean[installments_clean['payment_installments'] <= 24]

# ✅ Group and map
state_installments = installments_clean.groupby('customer_state')['payment_installments'].mean()
state_installments.index = state_installments.index.map(state_abbrev_to_name)

# ✅ Prepare DataFrame
state_installments_df = state_installments.reset_index()
state_installments_df.columns = ['State', 'Average Installments']
state_installments_df = state_installments_df.dropna()
state_installments_df['Average Installments'] = state_installments_df['Average Installments'].round(2)
state_installments_df = state_installments_df.sort_values('Average Installments', ascending=True)

# ✅ Horizontal Bar Chart
fig1 = px.bar(
    state_installments_df,
    x='Average Installments',
    y='State',
    orientation='h',
    title='<b>Average Number of Payment Installments by State</b>',
    labels={'Average Installments': 'Avg Installments', 'State': 'State'},
    color='Average Installments',
    color_continuous_scale='Plasma'
)
fig1.update_layout(
    width=900, height=600,
    title_font=dict(size=18, family='Arial', color='black'),
    plot_bgcolor='white',
    xaxis_title='<b>Average Installments</b>',
    yaxis_title='<b>State</b>',
    xaxis=dict(range=[1, 6]),
    xaxis_title_font=dict(color='black'),
    yaxis_title_font=dict(color='black')
)

fig1.show()


In [ ]:
# @title
import plotly.graph_objects as go

# Round to 2 decimals
state_installments_df['Average Installments'] = state_installments_df['Average Installments'].round(2)

fig3 = go.Figure()

# Add stems
for _, row in state_installments_df.iterrows():
    fig3.add_trace(go.Scatter(
        x=[1, row['Average Installments']],
        y=[row['State'], row['State']],
        mode='lines',
        line=dict(color='lightgray', width=1.5),
        showlegend=False,
        hoverinfo='skip'  # No hover on lines
    ))

# Add lollipops (markers only, no text outside)
fig3.add_trace(go.Scatter(
    x=state_installments_df['Average Installments'],
    y=state_installments_df['State'],
    mode='markers',
    marker=dict(size=10, color='crimson', line=dict(color='black', width=1)),
    hovertemplate='State: %{y}<br>Avg Installments: %{x:.2f}<extra></extra>',
    name=''
))

# Layout
fig3.update_layout(
    width=900,
    height=600,
    title='<b>Average Number of Payment Installments by State</b>',
    title_font=dict(size=18, family='Arial', color='black'),
    plot_bgcolor='white',
    showlegend=False,
    xaxis=dict(
        title='<b>Average Installments</b>',
        range=[1, 4.5],
        dtick= 0.5,
        showgrid=True,
        gridcolor='lightgray'
    ),
    yaxis=dict(
        title='<b>State</b>',
        showgrid=False
    ),
    xaxis_title_font=dict(color='black'),
    yaxis_title_font=dict(color='black')
)

fig3.show()


This scatter plot shows that higher payments don't neessarily mean higher number of installments.Most high-value purchases often occur in just one or two payments, as seen in the cluster of points on the left. **I used the log scale on the y-axis to compress extreme values and make it easier to see patterns across both small and large payment amounts.**

In [ ]:
# @title
import plotly.express as px

# Log-scaled scatter plot
fig = px.scatter(
    order_full,
    x='payment_installments',
    y='payment_value',
    title='<b>Payment Value vs. Number of Installments</b>',
    labels={
        'payment_installments': '<b>Number of Installments</b>',
        'payment_value': '<b>Payment Value (BRL)</b>'
    },
    hover_data=['payment_installments', 'payment_value'],
    opacity=0.3,
    color_discrete_sequence=['red']
)

# Apply log scale to y-axis
fig.update_yaxes(
    type='log',
    title='<b>Payment Value (BRL, Log Scale)</b>',
    title_font=dict(size=14, color='black')
)

# X-axis styling
fig.update_xaxes(
    title_font=dict(size=14, color='black')
)

# Title styling
fig.update_layout(
    title_font=dict(size=20, color='black', family='Arial'),
    width=800,
    height=600,
    plot_bgcolor='white',
    yaxis_gridcolor='lightgrey',
    xaxis_gridcolor='lightgrey'
)

fig.show()


This line chart shows how Brazilians paid for their online purchases from 2016 to 2018. **Credit cards** clearly dominated the total payment value, rising steadily and peaking sharply around **November 2017**, likely reflecting  **Black Friday**, which has become very popular in Brazil. **Boleto**, a local payment method, also shows smaller peaks but remains far below credit card totals. **Debit cards and vouchers** contributed only a tiny share across the period.



In [ ]:
# @title
import pandas as pd
import plotly.express as px

# Ensure datetime and sorting
order_full['order_purchase_timestamp'] = pd.to_datetime(order_full['order_purchase_timestamp'])
order_full['year_month'] = order_full['order_purchase_timestamp'].dt.to_period('M').astype(str)

# Group payment values by month and payment type
monthly_payment = (
    order_full.groupby(['year_month', 'payment_type'])['payment_value']
    .sum()
    .reset_index()
)

# Convert to datetime for proper sorting
monthly_payment['year_month'] = pd.to_datetime(monthly_payment['year_month'], format='%Y-%m')
monthly_payment = monthly_payment.sort_values('year_month')

# Plot the interactive line chart
fig = px.line(
    monthly_payment,
    x='year_month',
    y='payment_value',
    color='payment_type',
    markers=True,
    title='<b>Monthly Contribution of Each Payment Method (2016–2018)</b>',
    labels={
        'year_month': '<b>Month</b>',
        'payment_value': '<b>Total Payment Value (BRL)</b>',
        'payment_type': '<b>Payment Method</b>'
    },
    color_discrete_sequence=px.colors.qualitative.Dark24
)

# Update layout
fig.update_layout(
    width=850,
    height=450,
    plot_bgcolor='white',
    yaxis_gridcolor='lightgray',
    title_font=dict(size=18, family='Arial', color='black'),
    xaxis=dict(
        tickformat="%b %Y",  # Format like Jan 2017
        tickangle=45,
        tickmode="linear",   # Show all months
        dtick="M1"           # Tick every month
    ),
    yaxis=dict(
        title_font=dict(size=14, color='black'),
        tickformat=',',      # Add thousands separator to y-axis
    ),
    xaxis_title_font=dict(size=14, color='black'),
    legend_title_font=dict(size=12, color='black')
)

fig.show()


## **Delivery Performance Analysis**

This chart shows how early customers in each Brazilian state received their orders compared to the estimated delivery date. All states had orders delivered ahead of schedule on average. For example, customers in **Amapá** received their packages about 19 days early, while those in **Alagoas** received them around 10 days early.

The darker green bars at the bottom show the states with the earliest deliveries. This could mean that in these areas, the system adds extra time to the estimated delivery window just to be safe. Overall, it’s clear that most customers got their orders well before the promised date.

In [ ]:
# @title
import plotly.express as px

# 🧼 Drop duplicates: One row per order
clean_orders = order_full.drop_duplicates(subset='order_id')

# ✅ Drop NA delays and invalid states
valid_delays = clean_orders.dropna(subset=['delay', 'customer_state'])

# ✅ Clip out extreme delays
valid_delays = valid_delays[valid_delays['delay'].between(-30, 30)]

# ✅ Calculate average delay per state
avg_delay = (
    valid_delays.groupby('customer_state')['delay']
    .mean()
    .round(2)  # ✅ Round to 2 decimal places
    .rename_axis('State Abbr')
    .reset_index(name='Average Days Before Estimated')
)

# ✅ Map to full state names
avg_delay['State'] = avg_delay['State Abbr'].map(state_abbrev_to_name)

# ✅ Sort so darkest green is at the bottom (most early deliveries)
avg_delay = avg_delay.dropna().sort_values('Average Days Before Estimated', ascending=True)

# 📊 Plot
fig = px.bar(
    avg_delay,
    x='Average Days Before Estimated',
    y='State',
    orientation='h',
    color='Average Days Before Estimated',
    color_continuous_scale='Greens_r',
    title='<b>How Early Are Orders Delivered?</b>'
)

# 🎨 Layout and styling
fig.update_layout(
    width=950,
    height=600,
    plot_bgcolor='white',
    xaxis_title='<b>Average Days Before Estimated Delivery</b>',
    yaxis_title='<b>State</b>',
    title_font=dict(size=18, color='black'),
    xaxis_title_font=dict(color='black'),
    yaxis_title_font=dict(color='black'),
    coloraxis_colorbar=dict(title='Days Early', tickformat='.2f'),  # ✅ Format colorbar ticks
    xaxis=dict(tickformat='.2f')  # ✅ Format x-axis ticks
)

fig.show()


This scatterplot shows how long it took each order to arrive and whether it was early or late. The x-axis tells us the actual number of days it took for delivery, while the y-axis shows how early or late that order was compared to the estimated date.

Each dot represents one unique order — but only orders that meet certain conditions are shown. Specifically, the chart includes:

- Orders with delivery times between 0 and 40 days, and

- Delays between –20 and +20 days
This helps focus the analysis on realistic and typical deliveries, excluding extreme outliers that could distort the view.



Most of the dots fall below the zero line, meaning many orders were delivered early. In fact, a large cluster of orders arrived about 5 to 15 days ahead of schedule. This is a good sign of fast and efficient delivery.

However, there are also a few dots above the zero line, especially for deliveries that took longer (around 25 to 40 days). These are the late deliveries—orders that arrived after the promised date.

Overall, this chart shows that while a few orders were late, early delivery was much more common. It also suggests that longer delivery times increase the chance of delay, as the upper dots start to appear the further right you go.

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Outliers: orders excluded from the main scatterplot
outliers = valid_delays[
    (valid_delays['delay'] < -20) | (valid_delays['delay'] > 20) |
    (valid_delays['delivery_days'] > 40) | (valid_delays['delivery_days'] < 0)
]

# Main filtered data
main_orders = valid_delays[
    (valid_delays['delay'].between(-20, 20)) & (valid_delays['delivery_days'].between(0, 40))
]

# Add slight jitter to help with overlapping points
main_jitter_x = main_orders['delivery_days'] + np.random.uniform(-0.3, 0.3, size=len(main_orders))
main_jitter_y = main_orders['delay'] + np.random.uniform(-0.3, 0.3, size=len(main_orders))
outlier_jitter_x = outliers['delivery_days'] + np.random.uniform(-0.3, 0.3, size=len(outliers))
outlier_jitter_y = outliers['delay'] + np.random.uniform(-0.3, 0.3, size=len(outliers))

# Create side-by-side plots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "<b>Main Orders</b><br>(0–40 days, –20 to +20 delay)",
        "<b>Outlier Orders</b><br>Early by >20 days, Late by >20 days,<br>or Delivery >40 days or negative"
    ),
    shared_yaxes=True
)

# Main scatterplot
fig.add_trace(
    go.Scattergl(
        x=main_jitter_x,
        y=main_jitter_y,
        mode='markers',
        marker=dict(color='blue', opacity=0.1),
        name='Main Orders',
        hovertext=[
            f"Delivered in {d:.0f} days, {abs(l):.0f} days {'early' if l < 0 else 'late'}"
            for d, l in zip(main_orders['delivery_days'], main_orders['delay'])
        ],
        hoverinfo='text'
    ),
    row=1, col=1
)

# Outliers scatterplot
fig.add_trace(
    go.Scattergl(
        x=outlier_jitter_x,
        y=outlier_jitter_y,
        mode='markers',
        marker=dict(color='red', opacity=0.3),
        name='Outliers',
        hovertext=[
            f"Delivered in {d:.0f} days, {abs(l):.0f} days {'early' if l < 0 else 'late'}"
            for d, l in zip(outliers['delivery_days'], outliers['delay'])
        ],
        hoverinfo='text'
    ),
    row=1, col=2
)

# Layout and aesthetics
fig.update_layout(
    title=dict(
        text="<b>Delivery Time vs Delay: Main Orders vs Outliers</b>",
        x=0.5,
        xanchor='center',
        font=dict(size=26, family='Arial', color='black'),  # BLACK and BOLD
        y=0.97,
        yanchor='top'
    ),
    height=600,
    width=1000,
    plot_bgcolor='white',
    margin=dict(t=120),
    legend=dict(
        x=1.05,
        y=1,
        bgcolor='rgba(0,0,0,0)'
    )
)

# Subplot title fonts (black and consistent)
fig.update_annotations(font_size=16, font_color='black')

# Axis labels and tick font
fig.update_xaxes(
    title_text="<b>Total Days to Deliver (Actual)</b>",
    row=1, col=1,
    showgrid=True,
    title_font=dict(color='black'),
    tickfont=dict(color='black'),
    gridcolor='lightgray'
)
fig.update_xaxes(
    title_text="<b>Total Days to Deliver (Actual)</b>",
    row=1, col=2,
    showgrid=True,
    title_font=dict(color='black'),
    tickfont=dict(color='black'),
    gridcolor='lightgray'
)
fig.update_yaxes(
    title_text="<b>Days Early or Late</b>",
    row=1, col=1,
    range=[-60, 60],
    title_font=dict(color='black'),
    tickfont=dict(color='black'),
    gridcolor='lightgray'
)
fig.update_yaxes(
    range=[-60, 60],
    row=1, col=2,
    tickfont=dict(color='black'),
    gridcolor='lightgray'
)

fig.show()


In [ ]:
# @title
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Filter valid delay values within -20 to +20
hist_df = order_full.dropna(subset=['delay'])
hist_df = hist_df[hist_df['delay'].between(-20, 20)]

# Create histogram using numpy to precompute bin counts
counts, bins = np.histogram(hist_df['delay'], bins=30)
bin_centers = 0.5 * (bins[1:] + bins[:-1])

# Determine max y for annotations
max_count = max(counts)

# Manually construct histogram bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=bin_centers,
    y=counts,
    marker_color=['#2166ac' if x < 0 else '#b2182b' for x in bin_centers],
    hovertemplate='Delay: %{x} days<br>Orders: %{y:,}<extra></extra>',
    showlegend=False
))

# Layout formatting
fig.update_layout(
    title='Distribution of Delivery Delays',
    title_font=dict(size=18, color='black', family='Arial', weight='bold'),
    width=900,
    height=500,
    plot_bgcolor='white',
    xaxis=dict(
        title='Delivery Delay (days)',
        title_font=dict(size=14, color='black', family='Arial', weight='bold'),
        gridcolor='lightgray',
        zeroline=False
    ),
    yaxis=dict(
        title='Number of Orders',
        title_font=dict(size=14, color='black', family='Arial', weight='bold'),
        gridcolor='lightgray'
    )
)

# Add vertical dashed line at x=0
fig.add_vline(x=0, line_width=2, line_dash='dash', line_color='black')

# Add annotations
fig.add_annotation(
    x=0,
    y=max_count * 0.95,
    text="On-Time",
    showarrow=True,
    arrowhead=2,
    ax=40, ay=-30,
    font=dict(color='black', size=12)
)

fig.add_annotation(
    x=-15,
    y=max_count * 1,
    text="Delivered Early",
    showarrow=False,
    font=dict(color='black', size=12)
)

fig.add_annotation(
    x=15,
    y=max_count * 1,
    text="Delivered Late",
    showarrow=False,
    font=dict(color='black', size=12)
)

fig.show()


In [ ]:
# @title
import pandas as pd
import plotly.express as px

# --- STEP 1: Top 30 Sellers by Volume ---
top_sellers = order_items['seller_id'].value_counts().head(30).index
top_seller_orders = order_items[order_items['seller_id'].isin(top_sellers)]

# --- STEP 2: Merge with delay data ---
merged = pd.merge(
    top_seller_orders,
    order_full[['order_id', 'delay']],
    on='order_id',
    how='left'
)

# --- STEP 3: Compute average delay ---
avg_delay_by_seller = (
    merged.groupby('seller_id')['delay']
    .mean()
    .reset_index()
    .rename(columns={'seller_id': 'Seller', 'delay': 'Avg Delay (Days)'})
)

# --- STEP 4: Short Labels + Ranking ---
avg_delay_by_seller = avg_delay_by_seller.sort_values('Avg Delay (Days)').reset_index(drop=True)
avg_delay_by_seller['Label'] = avg_delay_by_seller['Seller'].str[:6]

# --- STEP 5: Plot ---
fig = px.bar(
    avg_delay_by_seller,
    x='Avg Delay (Days)',
    y='Label',
    orientation='h',
    color='Avg Delay (Days)',
    color_continuous_scale='Blues',
    title='<b>Average Delivery Delay by Top 30 Sellers (Ranked Best to Worst)</b>',
    labels={'Label': 'Seller'},
    height=600,
    width=850
)

# --- STEP 6: Add Text Labels on Bars ---
fig.update_traces(
    text=avg_delay_by_seller['Avg Delay (Days)'].round(1),
    textposition='outside',
    textfont=dict(color='black'),
    marker_line_color='black',
    marker_line_width=0.5
)

# --- STEP 7: Add Subtitle Annotation ---
fig.add_annotation(
    xref="paper", yref="paper",
    x=0, y=1.08,
    showarrow=False,
    text="<sub>All top 30 sellers consistently deliver earlier than the estimated date — by an average of 10–16 days.</sub>",
    font=dict(size=14, color='black')
)

# --- STEP 8: Label Top & Bottom Performers ---
# Most early (lowest value = last row)
most_early = avg_delay_by_seller.iloc[-1]
least_early = avg_delay_by_seller.iloc[0]

fig.add_annotation(
    x=most_early['Avg Delay (Days)'],
    y=most_early['Label'],
    text="⬅ Least Early",
    showarrow=False,
    xshift=50,
    font=dict(color='White', size=12)
)

fig.add_annotation(
    x=least_early['Avg Delay (Days)'],
    y=least_early['Label'],
    text="⬅ Most Early",
    showarrow=False,
    xshift=50,
    font=dict(color='red', size=12)
)

# --- STEP 9: Layout Styling ---
fig.update_layout(
    title={
        'text': '<b>Avg Delivery Delay by Top 30 Sellers (Ranked Best to Worst)</b>',
        'x': 0.5,            # 👈 Center horizontally
        'xanchor': 'center'  # 👈 Align title based on its center
    },
    title_font=dict(size=24, color='black'),
    xaxis_title='<b>Avg Delay (Days)</b>',
    yaxis_title='<b>Top 30 Sellers</b>',
    xaxis_title_font=dict(color='black'),
    yaxis_title_font=dict(color='black'),
    xaxis=dict(gridcolor='lightgray', tickformat=".1f"),
    yaxis=dict(gridcolor='lightgray'),
    font=dict(family='Arial', size=14, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=200, r=30, t=100, b=40),
    coloraxis_colorbar=dict(title='Avg Delay')
)

In [ ]:
# @title
# 🧹 Step 1: Prepare delay_by_cat_month
order_product = order_items.merge(products, on='product_id', how='left')
order_product = order_product.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']], on='order_id', how='left')
order_product = order_product.merge(category_translation, on='product_category_name', how='left')

# Convert to datetime
order_product['order_purchase_timestamp'] = pd.to_datetime(order_product['order_purchase_timestamp'])
order_product['order_delivered_customer_date'] = pd.to_datetime(order_product['order_delivered_customer_date'])
order_product['order_estimated_delivery_date'] = pd.to_datetime(order_product['order_estimated_delivery_date'])

# Compute delay and month
order_product['delay_days'] = (order_product['order_delivered_customer_date'] - order_product['order_estimated_delivery_date']).dt.days
order_product['year_month'] = order_product['order_purchase_timestamp'].dt.to_period("M").astype(str)

# Filter extreme outliers
order_product = order_product[(order_product['delay_days'] >= -20) & (order_product['delay_days'] <= 20)]

# Average delay by category and month
delay_by_cat_month = (
    order_product.groupby(['product_category_name_english', 'year_month'])['delay_days']
    .mean()
    .reset_index()
)


In [ ]:
# @title
# Step 2: Pivot for heatmap
delay_pivot = delay_by_cat_month.pivot_table(
    index='product_category_name_english',
    columns='year_month',
    values='delay_days'
)


In [ ]:
# @title
import plotly.express as px
import pandas as pd

# Convert 'year_month' to datetime for better axis spacing
delay_by_cat_month['year_month'] = pd.to_datetime(delay_by_cat_month['year_month'])

fig = px.imshow(
    delay_pivot.values,
    labels=dict(x="<b>Month</b>", y="<b>Product Category</b>", color="Delay (days)"),
    x=delay_pivot.columns,
    y=delay_pivot.index,
    color_continuous_scale='Viridis',
    aspect='auto'
)

fig.update_layout(
    title="<b>Average Delivery Delay by Product Category Over Time</b>",
    title_font=dict(size=22, color='black'),
    xaxis_tickformat="%b %Y",  # Format: Jan 2017
    xaxis_tickangle=45,
    width=900,
    height=600,
    plot_bgcolor='white',
    font=dict(size=12, color='black'),
    xaxis=dict(
        tickmode='linear',  # show every tick
        dtick="M1"          # every month
    )
)

fig.show()

In [ ]:
# @title
import plotly.express as px

import pandas as pd
import plotly.express as px

# Merge product items with product details
order_product = pd.merge(order_items, products, on='product_id', how='left')

# Merge with orders to get delivery and estimated dates
order_product = pd.merge(order_product, orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']], on='order_id', how='left')

# Merge with translation file to get English category names
order_product = pd.merge(order_product, category_translation, on='product_category_name', how='left')

# Convert to datetime
order_product['order_purchase_timestamp'] = pd.to_datetime(order_product['order_purchase_timestamp'])
order_product['order_delivered_customer_date'] = pd.to_datetime(order_product['order_delivered_customer_date'])
order_product['order_estimated_delivery_date'] = pd.to_datetime(order_product['order_estimated_delivery_date'])

# Calculate delay in days
order_product['delay_days'] = (order_product['order_delivered_customer_date'] - order_product['order_estimated_delivery_date']).dt.days

# Drop extreme outliers for a cleaner analysis
order_product = order_product[(order_product['delay_days'] > -20) & (order_product['delay_days'] < 20)]

# Group by product category (English name) and calculate average delay
avg_delay_per_category = order_product.groupby('product_category_name_english')['delay_days'].mean().sort_values()

# Select top 10 best (most early) delivered categories
best_10 = avg_delay_per_category.head(10).sort_values(ascending=True).reset_index()

# Sort and flip (so most early is on top)
best_10_sorted = best_10.sort_values('delay_days', ascending=True)


fig = px.bar(
    best_10_sorted,
    x='delay_days',
    y='product_category_name_english',
    orientation='h',
    text='delay_days',
    color='delay_days',
    color_continuous_scale='Blues',
    labels={
        'delay_days': 'Avg Early Delivery (days)',
        'product_category_name_english': 'Product Category'
    },
    title='<b>Top 10 Product Categories Delivered Earlier Than Estimated</b>'
)

# Style improvements
fig.update_traces(
    texttemplate='%{text:.1f} days',
    textposition='outside',
    marker_line_color='black',
    marker_line_width=0.5
)

fig.update_layout(
    xaxis_title='<b>Avg Early Delivery (Negative = Early)</b>',
    yaxis_title='',
    font=dict(family='Arial', size=14, color='black'),
    title_font=dict(size=22, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=400,
    width=800,
    margin=dict(l=180, r=30, t=60, b=40),
    coloraxis_showscale=False  # Remove color legend
)

fig.show()



3. Product Category Trends
Datasets: products, order_items, category_translation
* What are the most purchased categories?
* Which categories have the longest delivery times or highest dissatisfaction?
* Temporal changes in category popularity (monthly/yearly)

In [ ]:
# @title
import plotly.express as px

# Merge product and order item data
product_sales = order_items.merge(products, on='product_id', how='left')
product_sales = product_sales.merge(category_translation, on='product_category_name', how='left')

# Count purchases per category
most_purchased = product_sales['product_category_name_english'].value_counts().reset_index()
most_purchased.columns = ['product_category', 'num_purchases']

top15_purchased = most_purchased.head(15).sort_values(by='num_purchases', ascending=True)

fig = px.bar(
    top15_purchased,
    x='num_purchases',
    y='product_category',
    orientation='h',
    text='num_purchases',
    color_discrete_sequence=['#1f77b4'],  # A nice solid blue color
    labels={
        'product_category': 'Product Category',
        'num_purchases': 'Number of Items Sold'
    },
    title='<b>Top 15 Most Purchased Product Categories</b>'
)

fig.update_traces(
    texttemplate='%{text}',
    textposition='outside',
    marker_line_color='black',
    marker_line_width=0.5
)

fig.update_layout(
    xaxis_title='<b>Number of Items Sold</b>',
    yaxis_title='',
    font=dict(family='Arial', size=14, color='black'),
    title_font=dict(size=22, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    width=800,
    margin=dict(l=180, r=30, t=60, b=40)
)

fig.show()


In [ ]:
# @title
import pandas as pd
import plotly.graph_objects as go

# Prepare and sort data
most_purchased_sorted = most_purchased.head(15).sort_values(by='num_purchases', ascending=False).reset_index(drop=True)
most_purchased_sorted['cum_percent'] = most_purchased_sorted['num_purchases'].cumsum() / most_purchased_sorted['num_purchases'].sum() * 100

# Replace underscores with spaces and capitalize for better readability
most_purchased_sorted['product_category'] = most_purchased_sorted['product_category'].str.replace('_', ' ').str.title()


# Create Pareto chart
fig = go.Figure()

# Bar for number of purchases with value labels
fig.add_trace(go.Bar(
    x=most_purchased_sorted['product_category'],
    y=most_purchased_sorted['num_purchases'],
    name='Purchases',
    marker_color='steelblue',
    text=most_purchased_sorted['num_purchases'].apply(lambda x: f'{x:,}'),
    textposition='outside',
    textfont=dict(size=11, color='black'),
    hovertemplate='<b>%{x}</b><br>Purchases: %{y:,}<extra></extra>'
))

# Cumulative % line
fig.add_trace(go.Scatter(
    x=most_purchased_sorted['product_category'],
    y=most_purchased_sorted['cum_percent'],
    name='Cumulative %',
    mode='lines+markers',
    marker=dict(color='crimson', size=8),
    line=dict(width=2),
    yaxis='y2',
    hovertemplate='<b>%{x}</b><br>Cumulative: %{y:.1f}%<extra></extra>'
))

# Layout
fig.update_layout(
    title='<b>Pareto Chart of Top 15 Categories</b>',
    title_font=dict(size=22, family='Arial', color='black'),
    height=600,
    width=1000,
    plot_bgcolor='white',
    xaxis=dict(
    title='<b>Product Category</b>',
    tickangle=45,
    tickfont=dict(size=12, color='black', family='Arial Black'),  # Simulates bold
    title_font=dict(size=16, color='black'),
    showgrid=False,
    showline=True,
    linecolor='black'
),

    yaxis=dict(
        title='<b>Number of Purchases</b>',
        title_font=dict(size=16, color='black'),
        tickfont=dict(size=12, color='black'),
        gridcolor='lightgray',
        zeroline=False
    ),
    yaxis2=dict(
        title='<b>Cumulative %</b>',
        title_font=dict(size=16, color='black'),
        tickfont=dict(size=12, color='black'),
        overlaying='y',
        side='right',
        range=[0, 105],
        showgrid=False,
        showline=True,
        linecolor='black'
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
        font=dict(size=12)
    ),
    margin=dict(t=100, l=80, r=60, b=150)
)

fig.show()


In [ ]:
# @title
import pandas as pd
import plotly.express as px

# --- STEP 1: Merge necessary data ---
trend_data = order_items.merge(products, on='product_id', how='left')
trend_data = trend_data.merge(category_translation, on='product_category_name', how='left')
trend_data = trend_data.merge(orders[['order_id', 'order_purchase_timestamp']], on='order_id', how='left')

# --- STEP 2: Extract month ---
trend_data['order_purchase_timestamp'] = pd.to_datetime(trend_data['order_purchase_timestamp'])
trend_data['year_month'] = trend_data['order_purchase_timestamp'].dt.to_period('M').astype(str)

# --- STEP 3: Aggregate monthly orders by category ---
monthly_category_trend = (
    trend_data.groupby(['year_month', 'product_category_name_english'])['order_id']
    .count()
    .reset_index()
)
monthly_category_trend.columns = ['year_month', 'product_category', 'num_orders']

# --- STEP 4: Filter Top 5 categories ---
top5 = (
    monthly_category_trend.groupby('product_category')['num_orders']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist()
)
filtered_trend = monthly_category_trend[monthly_category_trend['product_category'].isin(top5)]

# --- STEP 5: Create full list of months for consistent x-axis ---
all_months = sorted(filtered_trend['year_month'].unique().tolist())

# --- STEP 6: Plot ---
fig = px.line(
    filtered_trend,
    x='year_month',
    y='num_orders',
    color='product_category',
    title='<b>Monthly Trend of Top 5 Product Categories</b>',
    labels={
        'year_month': 'Month',
        'num_orders': 'Number of Orders',
        'product_category': 'Product Category'
    },
    markers=True
)

# --- STEP 7: Styling traces ---
fig.update_traces(
    line=dict(width=1.5),
    marker=dict(size=4),
    hovertemplate='Category: %{fullData.name}<br>Month: %{x}<br>Orders: %{y}<extra></extra>'

)

# --- STEP 8: Layout tweaks ---
fig.update_layout(
    width=900,
    height=500,
    plot_bgcolor='white',
    title_font=dict(size=22, color='black', family='Arial'),

    xaxis=dict(
        title='<b>Month</b>',
        tickmode='array',
        tickvals=all_months,
        ticktext=all_months,
        tickangle=45,
        tickfont=dict(size=11, color='black'),
        title_font=dict(size=14, color='black'),
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False
    ),

    yaxis=dict(
        title='<b>Number of Orders</b>',
        tickfont=dict(size=12, color='black'),
        title_font=dict(size=14, color='black'),
        gridcolor='lightgray',
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=1
    ),

    legend=dict(
        title='',
        orientation='h',
        x=0,
        y=1.12,
        font=dict(size=12)
    ),

    margin=dict(l=60, r=40, t=100, b=80)
)

fig.show()


In [ ]:
# @title
# Merge delay data
delivery_data = order_items.merge(orders[['order_id', 'order_delivered_customer_date', 'order_purchase_timestamp']], on='order_id', how='left')
delivery_data = delivery_data.merge(products, on='product_id', how='left')
delivery_data = delivery_data.merge(category_translation, on='product_category_name', how='left')

# Compute delivery time
delivery_data['order_delivered_customer_date'] = pd.to_datetime(delivery_data['order_delivered_customer_date'])
delivery_data['order_purchase_timestamp'] = pd.to_datetime(delivery_data['order_purchase_timestamp'])
delivery_data['delivery_time_days'] = (delivery_data['order_delivered_customer_date'] - delivery_data['order_purchase_timestamp']).dt.days

# Average delivery time by category
avg_delivery_time = delivery_data.groupby('product_category_name_english')['delivery_time_days'].mean().sort_values(ascending=False).reset_index()

# Dissatisfaction analysis
dissatisfaction = reviews[reviews['review_score'] <= 2]
dissatisfaction = dissatisfaction.merge(order_items, on='order_id')
dissatisfaction = dissatisfaction.merge(products, on='product_id')
dissatisfaction = dissatisfaction.merge(category_translation, on='product_category_name', how='left')

# Count of low ratings by category
low_rating_counts = dissatisfaction['product_category_name_english'].value_counts().reset_index()
low_rating_counts.columns = ['product_category', 'low_ratings']


In [ ]:
# @title
# Round low ratings for display (technically not needed since they're integers, but for consistency)
low_rating_counts['label'] = low_rating_counts['low_ratings'].round(2)

fig2 = px.bar(
    low_rating_counts.head(10),
    x='product_category',
    y='low_ratings',
    title='<b>Top 10 Product Categories with Highest Dissatisfaction (Low Ratings)</b>',
    labels={
        'product_category': '<b>Product Category</b>',
        'low_ratings': '<b>Low Ratings</b>'
    },
    color='low_ratings',
    color_continuous_scale='Reds',
    text='label'
)

fig2.update_traces(
    textposition='outside',
    textfont_size=12
)

fig2.update_layout(
    width=900,
    height=500,
    plot_bgcolor='white',
    title_font=dict(size=20, color='black'),
    xaxis=dict(
        tickangle=45,
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black'),
        showgrid=False
    ),
    yaxis=dict(
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black'),
        gridcolor='lightgray'
    ),
    margin=dict(l=60, r=40, t=70, b=120)
)

fig2.show()


In [ ]:
# @title
import plotly.express as px

# ✅ Clean category labels for readability
avg_delivery_time = avg_delivery_time.copy()
avg_delivery_time['product_category_clean'] = avg_delivery_time['product_category_name_english'].str.replace('_', ' ').str.title()

# ✅ Round delivery time to 2 decimals for labeling
avg_delivery_time['label'] = avg_delivery_time['delivery_time_days'].round(2)

# ✅ Longest Delivery Times Chart
fig1 = px.bar(
    avg_delivery_time.head(10),
    x='product_category_clean',
    y='delivery_time_days',
    title='<b>Which Products Take the Longest to Arrive?</b>',
    labels={
        'product_category_clean': '<b>Product Category</b>',
        'delivery_time_days': '<b>Avg Delivery Time (days)</b>'
    },
    color='delivery_time_days',
    color_continuous_scale='OrRd',
    text='label'
)

# ✅ Text labels above bars
fig1.update_traces(
    textposition='outside',
    textfont_size=12
)

# ✅ Adjust y-axis range to avoid clipping
max_y = avg_delivery_time['delivery_time_days'].head(10).max()
fig1.update_yaxes(range=[0, max_y + 2])

# ✅ Layout formatting
fig1.update_layout(
    width=900,
    height=500,
    plot_bgcolor='white',
    title_font=dict(size=20, color='black'),
    xaxis=dict(
        tickangle=45,
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=14, color='black', family='Arial', weight='bold'),
        showgrid=False
    ),
    yaxis=dict(
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black'),
        gridcolor='lightgray'
    ),
    coloraxis_colorbar=dict(
        title=dict(
            text='Avg Delivery Time (days)',
            font=dict(color='black', size=14)
        ),
        tickfont=dict(color='black')
    ),
    margin=dict(l=60, r=20, t=120, b=60)
)

fig1.show()


In [ ]:
# @title
# Merge freight and payment data
freight_cost = order_items.merge(products, on='product_id', how='left')
freight_cost = freight_cost.merge(category_translation, on='product_category_name', how='left')
freight_cost = freight_cost.merge(orders[['order_id']], on='order_id')
freight_cost = freight_cost.merge(payments[['order_id', 'payment_value']], on='order_id', how='left')

# Scatter plot of freight vs payment
fig = px.scatter(freight_cost, x='freight_value', y='payment_value',
                 color='product_category_name_english',
                 title='<b>Freight Value vs. Payment Value by Product Category</b>',
                 labels={'freight_value': 'Freight Cost (R$)', 'payment_value': 'Total Payment (R$)'},
                 opacity=0.5)

fig.update_layout(plot_bgcolor='white', showlegend=True)
fig.show()


In [ ]:
# @title
import pandas as pd
import plotly.express as px

# Merge product and order item data
merged = order_items.merge(products, on='product_id', how='left')
merged = merged.merge(category_translation, on='product_category_name', how='left')

# Filter top 10 categories
top_categories = (
    merged['product_category_name_english']
    .value_counts()
    .head(10)
    .index
)
filtered = merged[merged['product_category_name_english'].isin(top_categories)]

# Optional: Create freight-to-price ratio
filtered['freight_ratio'] = filtered['freight_value'] / filtered['price']
filtered = filtered.replace([np.inf, -np.inf], np.nan).dropna(subset=['freight_value', 'price'])

# Plot
fig = px.scatter(
    filtered,
    x="freight_value",
    y="price",
    color="product_category_name_english",
    title="<b>🚚 Freight vs. Product Price (Top 10 Categories)</b>",
    labels={
        "freight_value": "<b>Freight (R$)</b>",
        "price": "<b>Product Price (R$)</b>",
        "product_category_name_english": "<b>Category</b>"
    },
    opacity=0.4,
    template="simple_white",
    color_discrete_sequence=px.colors.qualitative.Bold
)

fig.update_traces(marker=dict(size=6))

# Trendline overlay (optional but helpful if added category-wise separately)
# This works with px.scatter if category removed, or fit per category using statsmodels

fig.update_layout(
    title_font_size=20,
    legend_title_font=dict(size=13, family='Arial', color='black'),
    xaxis_title_font=dict(size=14, family='Arial', color='black'),
    yaxis_title_font=dict(size=14, family='Arial', color='black'),
    xaxis_tickfont=dict(size=12, family='Arial', color='black'),
    yaxis_tickfont=dict(size=12, family='Arial', color='black'),
    legend=dict(font=dict(size=12)),
    width=950,
    height=550
)

fig.show()


In [ ]:
# @title
import pandas as pd
import numpy as np
import plotly.express as px

# Clean again
order_df = order_items.merge(products, on='product_id', how='left')
order_df = order_df.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']], on='order_id', how='left')
order_df = order_df.merge(category_translation, on='product_category_name', how='left')

order_df['order_purchase_timestamp'] = pd.to_datetime(order_df['order_purchase_timestamp'])
order_df['order_delivered_customer_date'] = pd.to_datetime(order_df['order_delivered_customer_date'])
order_df['delivery_time_days'] = (order_df['order_delivered_customer_date'] - order_df['order_purchase_timestamp']).dt.days

# Filter valid values
df_clean = order_df.dropna(subset=['freight_value', 'delivery_time_days'])
df_clean = df_clean[(df_clean['delivery_time_days'] > 0) & (df_clean['delivery_time_days'] <= 30)]

# Bin freight into ranges
bins = [0, 25, 50, 75, 100, 150, 200, 400]
labels = ['0–25 R$', '26–50 R$', '51–75 R$', '76–100 R$', '101–150 R$', '151–200 R$', '200+ R$']
df_clean['freight_bin'] = pd.cut(df_clean['freight_value'], bins=bins, labels=labels, right=False)

# Drop missing bins (in case of NaN)
df_clean = df_clean.dropna(subset=['freight_bin'])

# Plot
fig = px.box(
    df_clean,
    x='freight_bin',
    y='delivery_time_days',
    title='<b>Does Paying More for Freight Mean Faster Delivery?</b><br><sub>Box plot of delivery time by freight cost bins</sub>',
    labels={
        'freight_bin': '<b>Freight Range</b>',
        'delivery_time_days': '<b>Delivery Time (Days)</b>'
    },
    template='simple_white'
)

# Add y-axis line and grid
fig.update_layout(
    width=900,
    height=500,
    title_font=dict(size=20, color='black'),
    xaxis_title_font=dict(size=14, color='black'),
    yaxis_title_font=dict(size=14, color='black'),
    xaxis_tickfont=dict(size=12, color='black'),
    yaxis_tickfont=dict(size=12, color='black'),
    plot_bgcolor='white',
    yaxis=dict(
        showline=True,
        linecolor='black',
        linewidth=1,
        showgrid=True,
        gridcolor='lightgray',
        griddash='solid'
    )
)

fig.show()


In [ ]:
# @title
# Convert timestamps to datetime
order_full['order_purchase_timestamp'] = pd.to_datetime(order_full['order_purchase_timestamp'])
order_full['order_delivered_customer_date'] = pd.to_datetime(order_full['order_delivered_customer_date'])

# Recreate delivery time column
order_full['delivery_time_days'] = (
    order_full['order_delivered_customer_date'] - order_full['order_purchase_timestamp']
).dt.days

# Clean data
df_clean = order_full.dropna(subset=['delivery_time_days', 'customer_state'])
df_clean = df_clean[(df_clean['delivery_time_days'] > 0) & (df_clean['delivery_time_days'] <= 30)]

# Create summary per state
state_delivery = df_clean.groupby('customer_state').agg({
    'delivery_time_days': 'mean',
    'order_id': 'count'
}).reset_index()

# Round for readability
state_delivery['delivery_time_days'] = state_delivery['delivery_time_days'].round(1)
state_delivery.columns = ['customer_state', 'delivery_time_days', 'order_count']


In [ ]:
# @title
import plotly.express as px
import json
import requests

# 📥 Load GeoJSON for Brazilian states
geojson_url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
brazil_geo = requests.get(geojson_url).json()

# ✅ Make sure your state names match the GeoJSON's feature names
state_name_mapping = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AM': 'Amazonas', 'AP': 'Amapá', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MG': 'Minas Gerais', 'MS': 'Mato Grosso do Sul', 'MT': 'Mato Grosso',
    'PA': 'Pará', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'PI': 'Piauí', 'PR': 'Paraná',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RO': 'Rondônia', 'RR': 'Roraima',
    'RS': 'Rio Grande do Sul', 'SC': 'Santa Catarina', 'SE': 'Sergipe', 'SP': 'São Paulo',
    'TO': 'Tocantins'
}

# 🧹 Map full state names for GeoJSON matching
state_delivery['state_name'] = state_delivery['customer_state'].map(state_name_mapping)

# 🗺️ Plot with GeoJSON and proper feature key
fig = px.choropleth(
    state_delivery,
    geojson=brazil_geo,
    locations='state_name',
    featureidkey='properties.name',  # key in the GeoJSON
    color='delivery_time_days',
    hover_name='state_name',
    color_continuous_scale='OrRd',
    title='<b>Average Delivery Time by Brazilian State</b><br><sub>Based on deliveries made in 30 days or less</sub>',
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title_font=dict(size=20, color='black'),
    coloraxis_colorbar=dict(title='Days'),
    geo=dict(projection_scale=2.5),
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()


In [ ]:
# @title
# STEP 1: Merge orders, order_items, and customers to build full order-level dataset
order_df = pd.merge(order_items, orders, on='order_id', how='left')
order_df = pd.merge(order_df, customers[['customer_id', 'customer_state']], on='customer_id', how='left')

# STEP 2: Calculate delivery_time_days if not already present
order_df['order_purchase_timestamp'] = pd.to_datetime(order_df['order_purchase_timestamp'])
order_df['order_delivered_customer_date'] = pd.to_datetime(order_df['order_delivered_customer_date'])
order_df['delivery_time_days'] = (order_df['order_delivered_customer_date'] - order_df['order_purchase_timestamp']).dt.days


In [ ]:
# @title
import plotly.express as px

# Map state codes to full state names
seller_state_map = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AM': 'Amazonas', 'AP': 'Amapá', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MG': 'Minas Gerais', 'MS': 'Mato Grosso do Sul', 'MT': 'Mato Grosso',
    'PA': 'Pará', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'PI': 'Piauí', 'PR': 'Paraná',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RO': 'Rondônia', 'RR': 'Roraima',
    'RS': 'Rio Grande do Sul', 'SC': 'Santa Catarina', 'SE': 'Sergipe', 'SP': 'São Paulo',
    'TO': 'Tocantins'
}

# Count sellers per state
sellers_state = sellers['seller_state'].value_counts().reset_index()
sellers_state.columns = ['seller_state', 'num_sellers']
sellers_state['state_name'] = sellers_state['seller_state'].map(seller_state_map)

# Load GeoJSON for Brazil states
import requests, json
geojson_url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
brazil_geo = requests.get(geojson_url).json()

# Plot choropleth
fig = px.choropleth(
    sellers_state,
    geojson=brazil_geo,
    locations='state_name',
    featureidkey='properties.name',
    color='num_sellers',
    hover_name='state_name',
    color_continuous_scale='Viridis_r',
    title='<b>Seller Concentration by Brazilian State</b>',
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title_font=dict(size=20, color='black'),
    coloraxis_colorbar=dict(title='Number of Sellers'),
    geo=dict(projection_scale=2.5),
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()


I plan to:
- uncover patterns on delivery times in different states.
- see if there is a delay in delivery for customers that pay with boleto
- do some sentiment analysis to see if customers were happy with some product categories and not happy about some.

In [ ]:
# @title
# ✅ Ensure datetime columns are in datetime format
order_full['order_delivered_customer_date'] = pd.to_datetime(order_full['order_delivered_customer_date'], errors='coerce')
order_full['order_purchase_timestamp'] = pd.to_datetime(order_full['order_purchase_timestamp'], errors='coerce')

# ✅ Create delivery_time_days if missing
if 'delivery_time_days' not in order_full.columns:
    order_full['delivery_time_days'] = (order_full['order_delivered_customer_date'] - order_full['order_purchase_timestamp']).dt.days


In [ ]:
# @title
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ✅ Step 1: Extract Year-Month
order_full['year_month'] = pd.to_datetime(order_full['order_purchase_timestamp']).dt.to_period("M").astype(str)

# ✅ Step 2: Filter valid delivery times
df_time = order_full.dropna(subset=['delivery_time_days', 'customer_state'])
df_time = df_time[(df_time['delivery_time_days'] > 0) & (df_time['delivery_time_days'] <= 30)]

# ✅ Step 3: Group by state and month to get average delivery time
state_time_trend = (
    df_time.groupby(['customer_state', 'year_month'])['delivery_time_days']
    .mean()
    .reset_index()
)

# ✅ Step 4: Map to full state names
state_time_trend['state_name'] = state_time_trend['customer_state'].map(state_abbrev_to_name)

# ✅ Step 5: Convert year_month to datetime for proper sorting
state_time_trend['year_month_dt'] = pd.to_datetime(state_time_trend['year_month'])

# ✅ Step 6: Get top 6 customer states by order volume
top_states = (
    order_full['customer_state'].value_counts()
    .head(6)
    .index.map(state_abbrev_to_name)
    .tolist()
)

# ✅ Step 7: Filter to top 6 states
state_time_trend = state_time_trend[state_time_trend['state_name'].isin(top_states)]

# ✅ Step 8: Create a full range of months for consistent x-axis ticks
all_months = pd.date_range(
    start=state_time_trend['year_month_dt'].min(),
    end=state_time_trend['year_month_dt'].max(),
    freq='MS'
)

# ✅ Step 9: Find the peak delivery time and annotate it
max_row = state_time_trend.loc[state_time_trend['delivery_time_days'].idxmax()]
max_date = max_row['year_month_dt']
max_value = max_row['delivery_time_days']
max_state = max_row['state_name']

# ✅ Step 10: Plot with custom tick formatting
fig = px.line(
    state_time_trend,
    x='year_month_dt',
    y='delivery_time_days',
    color='state_name',
    markers=True,
    labels={
        'year_month_dt': 'Month',
        'delivery_time_days': 'Avg Delivery Time (days)',
        'state_name': 'State'
    },
    title='<b>Delivery Performance Over Time</b><br><sub>Top 6 States by Order Volume</sub>'
)

fig.update_layout(
    title_font=dict(size=20, color='black'),
    font=dict(color='black'),
    xaxis=dict(
        title='<b>Month</b>',
        tickmode='array',
        tickvals=all_months,
        ticktext=[d.strftime('%b\n%Y') for d in all_months],
        tickangle=-45
    ),
    yaxis=dict(
        title='<b>Delivery Time (days)</b>'
    ),
    plot_bgcolor='white',
    xaxis_gridcolor='lightgray',
    yaxis_gridcolor='lightgray',
    width=900,
    height=550,
    annotations=[
        dict(
            x=max_date,
            y=max_value + 1,
            xref='x',
            yref='y',
            text=f"📍 Peak: {max_state} ({max_date.strftime('%b %Y')})",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(size=12, color='black'),
            bgcolor="white"
        )
    ]
)

fig.show()

## Review analysis

In [ ]:
# @title
import plotly.express as px

# Drop NaN and convert to string for categorical x-axis
filtered_reviews = order_full.dropna(subset=['review_score'])
filtered_reviews['review_score_str'] = filtered_reviews['review_score'].astype(int).astype(str)

# Get counts for labeling
review_counts = filtered_reviews['review_score_str'].value_counts().sort_index()

# Add text labels (value count per bar)
fig = px.histogram(
    filtered_reviews,
    x='review_score_str',
    title='<b>Distribution of Customer Review Scores</b>',
    labels={
        'review_score_str': 'Review Score',
        'count': 'Number of Reviews'
    },
    color_discrete_sequence=['teal'],
    text_auto=True  # Automatically places the count above bars
)

fig.update_layout(
    title_font=dict(size=20, color='black'),
    xaxis=dict(
        title='<b>Review Score</b>',
        tickmode='array',
        tickvals=['1', '2', '3', '4', '5'],
        tickfont=dict(size=12, color='black'),
        showline=True,
        linecolor='black',
        mirror=True
    ),
    yaxis=dict(
        title='<b>Number of Reviews</b>',
        tickfont=dict(size=12, color='black'),
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False,
        showline=True,
        linecolor='black',
        mirror=True
    ),
    font=dict(color='black'),
    plot_bgcolor='white',
    bargap=0.2,
    width=800,
    height=500
)

# Adjust text size and position
fig.update_traces(
    textfont_size=12,
    textposition='outside'
)

fig.show()


In [ ]:
# @title
import plotly.express as px

# Filter dataset
filtered_data = order_full[
    (order_full['review_score'].notna()) &
    (order_full['delivery_time_days'] <= 30)
]

fig = px.box(
    filtered_data,
    x='review_score',
    y='delivery_time_days',
    points=False,
    title='<b>Delivery Time vs. Review Score</b>',
    labels={
        'review_score': 'Review Score',
        'delivery_time_days': 'Delivery Time (days)'
    },
    color='review_score'
)

fig.update_layout(
    title_font=dict(size=20, color='black'),
    plot_bgcolor='white',
    font=dict(color='black'),
    showlegend=False,
    xaxis=dict(
        title=dict(text='<b>Review Score</b>'),
        type='category',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=14, color='black'),
        showline=True,
        linecolor='black',
        mirror=True
    ),
    yaxis=dict(
        title=dict(text='<b>Delivery Time (days)</b>'),
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=14, color='black'),
        showgrid=True,
        gridcolor='lightgray',
        showline=True,
        linecolor='black',
        mirror=True
    ),
    width=800,
    height=500
)

fig.show()


In [ ]:
# @title
import plotly.express as px
import plotly.graph_objects as go

# Compute repeat customer flag
repeat_counts = order_full.groupby('customer_unique_id')['order_id'].nunique()
order_full['is_repeat_customer'] = order_full['customer_unique_id'].map(lambda x: repeat_counts[x] > 1)

# Clean mapping for display
order_full['is_repeat_customer'] = order_full['is_repeat_customer'].map({True: 'Repeat Customer', False: 'New Customer'})

# Count review_score vs repeat status
review_counts = (
    order_full.groupby(['review_score', 'is_repeat_customer'])
    .size()
    .reset_index(name='count')
)

# Sort for consistent display
review_counts['review_score'] = review_counts['review_score'].astype(str)

# Custom color palette
color_map = {
    'New Customer': '#8c8c8c',        # medium gray
    'Repeat Customer': '#2ca02c'     # green
}

# Create grouped bar chart with labels
fig = px.bar(
    review_counts,
    x='review_score',
    y='count',
    color='is_repeat_customer',
    barmode='group',
    text='count',
    title='<b>Review Score Distribution: Repeat vs New Customers</b>',
    labels={
        'review_score': 'Review Score',
        'count': 'Number of Reviews',
        'is_repeat_customer': 'Customer Type'
    },
    color_discrete_map=color_map
)

# Update layout
fig.update_layout(
    title_font=dict(size=20, color='black'),
    plot_bgcolor='white',
    font=dict(color='black'),
    bargap=0.2,
    width=850,
    height=500,
    xaxis=dict(
        title=dict(text='<b>Review Score</b>'),
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=14, color='black'),
        showline=True,
        linecolor='black',
        mirror=True
    ),
    yaxis=dict(
        title=dict(text='<b>Number of Reviews</b>'),
        gridcolor='lightgray',
        tickfont=dict(size=12, color='black'),
        titlefont=dict(size=14, color='black'),
        showline=True,
        linecolor='black',
        mirror=True
    ),
    legend_title_text='<b>Customer Type</b>'
)

# Set text position for labels
fig.update_traces(textposition='outside', textfont_size=12)

fig.show()


In [ ]:
# @title
order_full['review_score'] = pd.to_numeric(order_full['review_score'], errors='coerce')

In [ ]:
# @title
import plotly.express as px

# ✅ Convert review_score back to numeric
order_full['review_score'] = pd.to_numeric(order_full['review_score'], errors='coerce')

# ✅ Group by seller, filter for meaningful volume
seller_reviews = (
    order_full.groupby('seller_id')
    .agg(avg_score=('review_score', 'mean'), num_reviews=('review_score', 'count'))
    .query('num_reviews >= 50')
    .sort_values('avg_score')
    .reset_index()
)

# ✅ Plot bottom 20 sellers
fig = px.bar(
    seller_reviews.head(20),
    x='avg_score',
    y='seller_id',
    orientation='h',
    color='avg_score',
    color_continuous_scale='OrRd',
    title='<b>Sellers with the Lowest Customer Satisfaction (Min. 50 Reviews)</b>',
    labels={'avg_score': 'Avg Review Score', 'seller_id': 'Seller ID'}
)

fig.update_layout(yaxis=dict(autorange="reversed"), plot_bgcolor='white')
fig.show()


In [ ]:
# @title
# Re-merge reviews with comment message included
order_full = pd.merge(
    order_full,
    reviews[['order_id', 'review_comment_message']],
    on='order_id',
    how='left'
)

In [ ]:
# @title
# Recalculate seller review stats (exclude those with very few reviews)
seller_reviews = (
    order_full.groupby('seller_id')
    .agg(avg_score=('review_score', 'mean'), num_reviews=('review_score', 'count'))
    .query('num_reviews >= 50')
    .sort_values('avg_score')
)

# Get bottom N sellers — say, 5
low_rated_sellers = seller_reviews.head(5).index.tolist()


In [ ]:
# @title
# ✅ Re-merge with full reviews table
order_full = pd.merge(
    order_full.drop(columns=['review_score'], errors='ignore'),  # Avoid duplicates
    reviews[['order_id', 'review_score', 'review_comment_message']],
    on='order_id',
    how='left'
)

In [ ]:
# @title
print('review_comment_message' in order_full.columns)

In [ ]:
# @title
order_full = order_full.drop(columns=['review_comment_message'], errors='ignore')

order_full = pd.merge(
    order_full,
    reviews[['order_id', 'review_comment_message']],
    on='order_id',
    how='left'
)

In [ ]:
# @title
low_seller_reviews = order_full[
    (order_full['seller_id'].isin(low_rated_sellers)) &
    (order_full['review_comment_message'].notna())
].copy()

In [ ]:
# @title
import re

# ✅ Clean text: lowercase, remove punctuation/numbers/symbols
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # keep only letters
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    return text

low_seller_reviews['clean_comment'] = low_seller_reviews['review_comment_message'].apply(clean_text)
low_seller_reviews[['review_comment_message', 'clean_comment']].head()


In [ ]:
# @title
from collections import Counter

# ✅ Combine all cleaned reviews into one big string
all_text = ' '.join(low_seller_reviews['clean_comment'].dropna())

# ✅ Count word frequencies
word_counts = Counter(all_text.split())

# ✅ Show top 20 most common complaint words
common_words = word_counts.most_common(20)
for word, count in common_words:
    print(f"{word}: {count}")


In [ ]:
# @title
import plotly.express as px
import pandas as pd

# Sample data
top_words = {
    'word': ['não', 'o', 'que', 'produto', 'de', 'a', 'e', 'cor', 'me', 'recebi', 'um', 'ou', 'esta', 'comprei', 'chegou', 'j', 'loja', 'mas', 'meu', 's'],
    'count': [516, 502, 402, 375, 354, 343, 343, 294, 231, 228, 213, 210, 206, 184, 157, 154, 151, 148, 135, 135]
}

# English translations
translations = {
    'não': 'no/not', 'o': 'the', 'que': 'that/what', 'produto': 'product', 'de': 'of/from',
    'a': 'to', 'e': 'and', 'cor': 'color', 'me': 'me', 'recebi': 'received',
    'um': 'a/an', 'ou': 'or', 'esta': 'this/is', 'comprei': 'I bought', 'chegou': 'arrived',
    'j': '[unknown]', 'loja': 'store', 'mas': 'but', 'meu': 'my', 's': '[unknown]'
}

# Convert to DataFrame
df = pd.DataFrame(top_words)
df['translation'] = df['word'].map(translations)

# Custom hover text
df['hover'] = (
    'Portuguese: ' + df['word'] +
    '<br>English: ' + df['translation'] +
    '<br>Frequency: ' + df['count'].astype(str)
)

# Plotly bar chart
fig = px.bar(
    df,
    x='count',
    y='word',
    orientation='h',
    color='count',
    color_continuous_scale='Reds',
    title='<b>Top 20 Words in Low-Rated Seller Reviews</b>',
    labels={'word': 'Portuguese Word', 'count': 'Frequency'},
    hover_name='word',
    custom_data=['hover']
)

# Update hover template
fig.update_traces(
    hovertemplate='%{customdata[0]}<extra></extra>'
)

fig.update_layout(
    title_font=dict(size=20, color='black'),
    plot_bgcolor='white',
    font=dict(color='black'),
    yaxis=dict(autorange="reversed")
)

fig.show()


In [ ]:
# @title
from wordcloud import WordCloud

wordcloud = WordCloud(
    width=600,
    height=400,
    background_color='white',
    colormap='Blues'
).generate(all_text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Frequent Words in Low-Rated Seller Reviews', fontsize=16, weight='bold')
plt.show()


In [ ]:
# @title
# Subset for positive and negative reviews (with comment text)
pos_reviews = order_full[(order_full['review_score'] >= 4) & order_full['review_comment_message'].notna()]
neg_reviews = order_full[(order_full['review_score'] <= 2) & order_full['review_comment_message'].notna()]


In [ ]:
# @title
# Clean text function (assumed to be defined already)
def clean_text(text):
    text = re.sub(r'[^a-zA-ZÀ-ÿ]', ' ', text)  # keep only letters
    text = text.lower().strip()
    return text

# Apply cleaning
pos_reviews['clean_comment'] = pos_reviews['review_comment_message'].apply(clean_text)
neg_reviews['clean_comment'] = neg_reviews['review_comment_message'].apply(clean_text)

In [ ]:
# @title
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all text into one string
pos_text = ' '.join(pos_reviews['clean_comment'].dropna())
neg_text = ' '.join(neg_reviews['clean_comment'].dropna())

# Create side-by-side word clouds
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Positive
wordcloud_pos = WordCloud(
    width=800, height=800, background_color='white',
    colormap='Greens', stopwords=None
).generate(pos_text)

ax[0].imshow(wordcloud_pos, interpolation='bilinear')
ax[0].set_title('Positive Reviews (Score ≥ 4)', fontsize=20)
ax[0].axis('off')

# Negative
wordcloud_neg = WordCloud(
    width=800, height=800, background_color='white',
    colormap='Reds', stopwords=None
).generate(neg_text)

ax[1].imshow(wordcloud_neg, interpolation='bilinear')
ax[1].set_title('Negative Reviews (Score ≤ 2)', fontsize=20)
ax[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# @title
!pip install -q spacy spacytextblob
!python -m spacy download pt_core_news_sm

In [ ]:
# @title
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load Portuguese model
nlp = spacy.load('pt_core_news_sm')
nlp.add_pipe('spacytextblob')

In [ ]:
# @title
def get_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return None
    doc = nlp(text)
    return doc._.blob.polarity

In [ ]:
# @title
def classify_sentiment(score):
    if score is None:
        return 'unknown'
    elif score > 0.05:
        return 'positive'
    elif score < -0.05:
        return 'negative'
    else:
        return 'neutral'

low_seller_reviews['sentiment'] = low_seller_reviews['review_comment_message'].apply(get_sentiment)
low_seller_reviews['sentiment_label'] = low_seller_reviews['sentiment'].apply(classify_sentiment)


In [ ]:
# @title
print("🟥 NEGATIVE REVIEWS")
print(low_seller_reviews[low_seller_reviews['sentiment_label'] == 'negative']['review_comment_message'].dropna().sample(5, random_state=10).to_string(index=False))

print("\n🟩 POSITIVE REVIEWS")
print(low_seller_reviews[low_seller_reviews['sentiment_label'] == 'positive']['review_comment_message'].dropna().sample(5, random_state=11).to_string(index=False))


In [ ]:
# @title
!pip install deep-translator

In [ ]:
# @title
# 🧠 Import the translator
from deep_translator import GoogleTranslator

# 🔴 NEGATIVE REVIEWS in Portuguese
negative_reviews = [
    "Um verdadeiro absurdo hj já são 09 de agosto e ...",
    "Comprei a base dermacol cor 207, pela cor mostr...",
    "O cartão sd apresentou problemas e não consigo ...",
    "As toalhas muito ruins , finas",
    "Já faz algum tempo que a compra ocorreu e ainda..."
]

# 🟢 POSITIVE REVIEWS in Portuguese
positive_reviews = [
    "O produto tem um preço legal, mas não entregaram...",
    "Enviaram o produto errado pra mim!",
    "RECEBI UM PRODUTO F-A-L-S-I-F-I-C-A-D-O. Ñ SABE...",
    "O relógio chegou sem funcionar, e não é original...",
    "Recebi um relogio falsificado. Vou devolver e q..."
]

# ✅ Translate each list
neg_en = [GoogleTranslator(source='pt', target='en').translate(text) for text in negative_reviews]
pos_en = [GoogleTranslator(source='pt', target='en').translate(text) for text in positive_reviews]

# 📋 Display the results
print("🟥 NEGATIVE REVIEWS (🇧🇷 Portuguese + 🇺🇸 English)\n")
for pt, en in zip(negative_reviews, neg_en):
    print(f"🇧🇷 {pt}\n🇺🇸 {en}\n")

print("\n🟩 POSITIVE REVIEWS (🇧🇷 Portuguese + 🇺🇸 English)\n")
for pt, en in zip(positive_reviews, pos_en):
    print(f"🇧🇷 {pt}\n🇺🇸 {en}\n")


In [ ]:
# @title
!pip install deep-translator
!pip install textblob

In [ ]:
# @title
# ✅ Install required packages (uncomment if needed)
# !pip install deep-translator textblob plotly

from deep_translator import GoogleTranslator
from textblob import TextBlob
import pandas as pd
import plotly.express as px

# 🔴 Your review sentences (Portuguese)
reviews = [
    "Um verdadeiro absurdo hj já são 09 de agosto e ...",
    "As toalhas muito ruins , finas",
    "O cartão sd apresentou problemas e não consigo ...",
    "As toalhas muito ruins , finas",
    "Já faz algum tempo que a compra ocorreu e ainda...",
    "O produto tem um preço legal, mas não entregaram...",
    "Enviaram o produto errado pra mim!\r\nNa loja v...",
    "RECEBI UM PRODUTO F-A-L-S-I-F-I-C-A-D-O. Ñ SABE...",
    "O relógio chegou sem funcionar, e não é origina...",
    "Recebi um relogio falsificado. Vou devolver e q..."
]

# ✅ Translate + Sentiment scoring
translated = []
polarities = []
labels = []

for text in reviews:
    try:
        en = GoogleTranslator(source='auto', target='en').translate(text)
        translated.append(en)
        polarity = TextBlob(en).sentiment.polarity
        polarities.append(round(polarity, 2))
        if polarity > 0.1:
            labels.append("🟩 Positive")
        elif polarity < -0.1:
            labels.append("🟥 Negative")
        else:
            labels.append("⬜ Neutral")
    except:
        translated.append("Translation failed")
        polarities.append(None)
        labels.append("❌ Error")

# ✅ Build DataFrame
df_sentiment = pd.DataFrame({
    "Original (PT)": reviews,
    "Translated (EN)": translated,
    "Sentiment Score": polarities,
    "Sentiment Label": labels
})

# ✅ Display table
import IPython.display as display
display.display(df_sentiment)

# ✅ Prepare data for Plotly
sentiment_counts = df_sentiment["Sentiment Label"].value_counts().reindex(
    ["🟥 Negative", "⬜ Neutral", "🟩 Positive"]
).reset_index()
sentiment_counts.columns = ["Sentiment Label", "Number of Reviews"]

# ✅ Plot bar chart with slim bars and bold axis titles
fig = px.bar(
    sentiment_counts,
    x='Sentiment Label',
    y='Number of Reviews',
    title='<b>Sentiment Distribution of Reviews</b>',
    color='Sentiment Label',
    color_discrete_map={
        "🟥 Negative": "red",
        "⬜ Neutral": "gray",
        "🟩 Positive": "green"
    }
)